In [ ]:
# Valeurs pour l'augmentation progressive de la taille de l'ensemble d'entraînement
train_ratio_start = 0.2  # Commence avec 20% des données
train_ratio_end = 1.0    # Termine avec 100% des données

train_ratio = train_ratio_start + (train_ratio_end - train_ratio_start) * (epoch / max(1, n_epochs-1))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import joblib
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, log_loss
from sklearn.model_selection import train_test_split, RandomizedSearchCV, learning_curve
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.exceptions import ConvergenceWarning
import time
from tqdm import tqdm
import warnings
from training_code.utils import load_and_preprocess_data, plot_learning_curve,plot_training_metrics


# Supprimer les avertissements non nécessaires
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=ConvergenceWarning)

# Création des dossiers pour les résultats
os.makedirs('figures/xgb', exist_ok=True)
os.makedirs('models', exist_ok=True)


In [ ]:

def optimize_xgb_hyperparameters(X_train, y_train, X_val, y_val, cv=3):
    """
    Optimise les hyperparamètres du modèle XGBoost
    Args:
        X_train: Caractéristiques d'entraînement
        y_train: Étiquettes d'entraînement
        X_val: Caractéristiques de validation
        y_val: Étiquettes de validation
        cv: Nombre de plis pour la validation croisée
    Returns:
        Meilleurs hyperparamètres et score
    """
    print(f"Optimisation des hyperparamètres XGBoost...")
    param_dist = {
        'n_estimators': [50, 100, 150],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.1, 0.2],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0],
        'gamma': [0, 0.1, 0.2],
        'reg_alpha': [0, 0.1, 0.2],
        'reg_lambda': [1, 1.5, 2]
    }

    xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
    grid_search = RandomizedSearchCV(
        xgb, param_distributions=param_dist, n_iter=15, cv=cv, scoring='accuracy', n_jobs=-1, verbose=1
    )

    start_time = time.time()
    grid_search.fit(X_train, y_train)
    search_time = time.time() - start_time
    print(f"Recherche par grille terminée en {search_time:.2f} secondes")

    val_score = accuracy_score(y_val, grid_search.predict(X_val))
    print(f"Meilleurs hyperparamètres: {grid_search.best_params_}")
    print(f"Score de validation croisée: {grid_search.best_score_:.4f}")
    print(f"Score sur l'ensemble de validation: {val_score:.4f}")

    return grid_search.best_params_, val_score




In [ ]:

def train_xgb_progressive(X_train, y_train, X_val, y_val, X_test, y_test, best_params, n_epochs=25):
    """
    Entraîne le modèle XGBoost de manière progressive en augmentant la taille de l'ensemble d'entraînement
    Args:
        X_train: Caractéristiques d'entraînement
        y_train: Étiquettes d'entraînement
        X_val: Caractéristiques de validation
        y_val: Étiquettes de validation
        X_test: Caractéristiques de test
        y_test: Étiquettes de test
        best_params: Meilleurs hyperparamètres trouvés
        n_epochs: Nombre d'époques d'entraînement
    Returns:
        Historique des métriques et meilleur modèle
    """
    print(f"Entraînement progressif du XGBoost sur {n_epochs} époques...")

    # Métriques
    train_accuracies = []
    val_accuracies = []
    train_precisions = []
    val_precisions = []
    train_recalls = []
    val_recalls = []
    train_f1s = []
    val_f1s = []
    train_losses = []
    val_losses = []
    epoch_train_sizes = []
     # Convertir en tableaux NumPy pour éviter les problèmes d'indexation
    if not isinstance(X_train, np.ndarray):
        X_train = np.array(X_train)
    if not isinstance(y_train, np.ndarray):
        y_train = np.array(y_train)
    if not isinstance(X_val, np.ndarray):
        X_val = np.array(X_val)
    if not isinstance(y_val, np.ndarray):
        y_val = np.array(y_val)
    if not isinstance(X_test, np.ndarray):
        X_test = np.array(X_test)
    if not isinstance(y_test, np.ndarray):
        y_test = np.array(y_test)

    # Initialiser le modèle avec les meilleurs paramètres
    model = XGBClassifier(**best_params, use_label_encoder=False, eval_metric='mlogloss', random_state=42)

    # Meilleur modèle
    best_model = None
    best_val_acc = 0

    # Valeurs pour l'augmentation progressive de la taille de l'ensemble d'entraînement
    train_ratio_start = 0.2  # Commence avec 20% des données
    train_ratio_end = 1.0    # Termine avec 100% des données

    # Assurer une répartition équilibrée des classes lors de l'échantillonnage
    class_indices = {}
    unique_classes = np.unique(y_train)
    for cls in unique_classes:
        class_indices[cls] = np.where(y_train == cls)[0]

    with tqdm(total=n_epochs, desc="Entraînement") as pbar:
        for epoch in range(n_epochs):
            try:
                # Augmentation progressive de la taille de l'entraînement
                train_ratio = train_ratio_start + (train_ratio_end - train_ratio_start) * (epoch / max(1, n_epochs-1))
                indices = []
                for cls in unique_classes:
                    n_samples = int(len(class_indices[cls]) * train_ratio)
                    cls_sample = np.random.choice(class_indices[cls], n_samples, replace=False)
                    indices.extend(cls_sample)
                np.random.shuffle(indices)
                train_size = len(indices)
                epoch_train_sizes.append(train_size)
                # Extraire les données d'entraînement pour cette époque
                X_epoch = X_train[indices]
                y_epoch = y_train[indices]

                # Créer et entraîner le modèle avec les meilleurs hyperparamètres
                model.fit(X_epoch, y_epoch)

                # Évaluations
                train_pred = model.predict(X_epoch)
                val_pred = model.predict(X_val)

                train_accuracies.append(accuracy_score(y_epoch, train_pred))
                val_accuracies.append(accuracy_score(y_val, val_pred))
                train_precisions.append(precision_score(y_epoch, train_pred, zero_division=0))
                val_precisions.append(precision_score(y_val, val_pred, zero_division=0))
                train_recalls.append(recall_score(y_epoch, train_pred, zero_division=0))
                val_recalls.append(recall_score(y_val, val_pred, zero_division=0))
                train_f1s.append(f1_score(y_epoch, train_pred, zero_division=0))
                val_f1s.append(f1_score(y_val, val_pred, zero_division=0))

                # Calcul des pertes (log loss) si predict_proba est disponible
                if hasattr(model, 'predict_proba'):
                    try:
                        train_probs = model.predict_proba(X_epoch)
                        val_probs = model.predict_proba(X_val)
                        # Vérifier la validité des probabilités
                        if not np.any(np.isnan(train_probs)) and not np.any(np.isnan(val_probs)):
                            train_loss = log_loss(y_epoch, train_probs)
                            val_loss = log_loss(y_val, val_probs)
                        else:
                            train_loss = -np.log(max(0.001, train_accuracies[-1]))
                            val_loss = -np.log(max(0.001, val_accuracies[-1]))
                    except Exception:
                        # En cas d'erreur, utiliser une approximation
                        train_loss = -np.log(max(0.001, train_accuracies[-1]))
                        val_loss = -np.log(max(0.001, val_accuracies[-1]))
                else:
                    # Si predict_proba n'est pas disponible, simuler une relation inverse avec l'accuracy
                    train_loss = -np.log(max(0.001, train_accuracies[-1]))
                    val_loss = -np.log(max(0.001, val_accuracies[-1]))

                train_losses.append(train_loss)
                val_losses.append(val_loss)

                pbar.update(1)
                pbar.set_postfix({
                    'Train Acc': f'{train_accuracies[-1]:.4f}',
                    'Val Acc': f'{val_accuracies[-1]:.4f}',
                    'Train Size': train_size
                })

                # Suivre le meilleur modèle
                if val_accuracies[-1] > best_val_acc:
                    best_val_acc = val_accuracies[-1]
                    best_model = model

            except Exception as e:
                print(f"Erreur à l'époque {epoch+1}: {str(e)}")
                continue

    # Si aucun modèle valide n'a été trouvé, utiliser un modèle par défaut
    if best_model is None:
        print("Aucun modèle valide trouvé pendant l'entraînement. Création d'un modèle par défaut.")
        best_model = XGBClassifier(**best_params, use_label_encoder=False, eval_metric='mlogloss', random_state=42)
        best_model.fit(X_train, y_train)

    try:
        plot_training_metrics(
            train_accuracies, val_accuracies, train_losses, val_losses, 
            train_f1s, val_f1s, train_recalls, val_recalls, n_epochs,
            algorithm_name="XGBoost", output_dir="figures/xgb"
        )
    except Exception as e:
        print(f"Erreur lors de la création des graphiques: {str(e)}")

    # Évaluation finale du meilleur modèle sur l'ensemble de test
    test_pred = best_model.predict(X_test)
    test_accuracy = accuracy_score(y_test, test_pred)
    test_precision = precision_score(y_test, test_pred, zero_division=0)
    test_recall = recall_score(y_test, test_pred, zero_division=0)
    test_f1 = f1_score(y_test, test_pred, zero_division=0)
    conf_matrix = confusion_matrix(y_test, test_pred)

    # Sauvegarder
    joblib.dump(best_model, "models/xgb_best.pkl")
    print("✅ Meilleur modèle XGBoost sauvegardé")

    return {
        'model': best_model,
        'train_accuracies': train_accuracies,
        'val_accuracies': val_accuracies,
        'test_accuracy': test_accuracy,
        'test_precision': test_precision,
        'test_recall': test_recall,
        'test_f1': test_f1,
        'confusion_matrix': conf_matrix
    }



In [ ]:

def plot_training_size(epoch_train_sizes, n_epochs):
    """
    Trace la progression de la taille de l'ensemble d'entraînement
    Args:
        epoch_train_sizes: Liste des tailles d'entraînement à chaque époque
        n_epochs: Nombre d'époques
    """
    plt.figure(figsize=(10, 5))
    plt.plot(range(1, n_epochs+1), epoch_train_sizes, '-o', linewidth=2, markersize=4, color='#2ca02c')
    plt.title('Progression de la taille de l\'ensemble d\'entraînement', fontsize=14, fontweight='bold')
    plt.xlabel('Époque', fontsize=12)
    plt.ylabel('Nombre d\'échantillons', fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.savefig('figures/xgb/xgb_training_size.png', dpi=300)
    plt.close()



In [ ]:

def plot_confusion_matrix(conf_matrix):
    """
    Trace la matrice de confusion
    Args:
        conf_matrix: Matrice de confusion
    """
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Normal', 'Attack'], yticklabels=['Normal', 'Attack'])
    plt.title('Matrice de confusion (Ensemble de test)', fontsize=14, fontweight='bold')
    plt.xlabel('Classe prédite', fontsize=12)
    plt.ylabel('Classe réelle', fontsize=12)
    plt.tight_layout()
    plt.savefig('figures/xgb/xgb_confusion_matrix.png', dpi=300)
    plt.close()



In [ ]:

def main(data_path="UNSW_NB15_training-set.csv", test_size=0.2, val_size=0.15, n_epochs=25, random_state=42):
    """
    Fonction principale qui exécute tout le pipeline
    """
    print("=" * 80)
    print("Pipeline d'analyse et d'entraînement XGBoost pour la détection d'intrusion réseau")
    print("=" * 80)

    start_time = time.time()
    

    X_train, X_val, X_test, y_train, y_val, y_test, scaler, encoders = load_and_preprocess_data(
        filepath=data_path, test_size=test_size, val_size=val_size, random_state=random_state
    )
    joblib.dump(scaler, "models/scaler_xgb.pkl")
    joblib.dump(encoders, "models/label_encoders_xgb.pkl")

    # Optimiser les hyperparamètres
    best_params, val_score = optimize_xgb_hyperparameters(X_train, y_train, X_val, y_val, cv=3)
    # Tracer la courbe d'apprentissage pour évaluer l'impact de la taille de l'ensemble d'entraînement
    model = XGBClassifier(**best_params, use_label_encoder=False, eval_metric='mlogloss', random_state=42)
    plot_learning_curve(model, X_train, y_train)
    # Entraînement progressif
    results = train_xgb_progressive(X_train, y_train, X_val, y_val, X_test, y_test, best_params, n_epochs=n_epochs)

    elapsed_time = time.time() - start_time
    print(f"\nTemps total d'exécution: {elapsed_time:.2f} secondes ({elapsed_time / 60:.2f} minutes)")
    print("\n📊 Résumé des performances:")
    print(f"Accuracy finale sur le test: {results['test_accuracy']:.4f}")
    print(f"Precision finale sur le test: {results['test_precision']:.4f}")
    print(f"Recall final sur le test: {results['test_recall']:.4f}")
    print(f"F1-Score final sur le test: {results['test_f1']:.4f}")
    print(f"Meilleurs hyperparamètres: {best_params}")

    # Évaluation finale du modèle
    print("🔍 Analyse de la matrice de confusion:")
    conf_matrix = results['confusion_matrix']
    tn, fp, fn, tp = conf_matrix.ravel()
    total = tn + fp + fn + tp
    print(f" - Vrais Négatifs (TN): {tn} ({tn/total*100:.2f}%)")
    print(f" - Faux Positifs (FP): {fp} ({fp/total*100:.2f}%)")
    print(f" - Faux Négatifs (FN): {fn} ({fn/total*100:.2f}%)")
    print(f" - Vrais Positifs (TP): {tp} ({tp/total*100:.2f}%)")
    print(f" - Taux de faux positifs: {fp/(fp+tn)*100:.2f}%")
    print(f" - Taux de faux négatifs: {fn/(fn+tp)*100:.2f}%")


    # Tracer la matrice de confusion pour le meilleur modèle
    plot_confusion_matrix(conf_matrix)


if __name__ == "__main__":
    main()